In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
from pathlib import Path
import re
from collections import defaultdict
from typing import Optional

#Настройки отображения
pd.options.display.max_columns = None
pd.options.display.max_rows = 100

In [ ]:
# Загрузка сырых данных
file_paths = [
    'Путь 1',
    'Путь 2',
    'Путь 3',
    'Путь 4',

]

df_enter = pd.concat([pd.read_excel(file) for file in file_paths], ignore_index=True)

# I. Формирование справочника SKU

In [ ]:
# Отбираем необходимые поля для анализа в отдельный DataFrame
df_for_work = df_enter[['Клиент (вх)', 'Бар-код', 'Номенклатура (вх)', 'Подгруппа товара (вх)', 
                         'Поставщик РЦ', 'Тип поставщика РЦ', 'Производитель', 'Продажи, шт.', 
                         'Продажи в руб.', 'Себестоимость в руб.']]

# Генерируем новые уникальное поле (ключ)

# Устраняем возможные ошибки в данных (лишние пробелы)
df_for_work['Клиент (вх)'] = df_for_work['Клиент (вх)'].str.strip()
df_for_work['Бар-код'] = df_for_work['Бар-код'].str.strip()

df_for_work['Клиент_БарКод'] = df_for_work['Клиент (вх)'].str.cat(df_for_work['Бар-код'], sep='_')
df_for_work.insert(0, 'Клиент_БарКод', df_for_work.pop('Клиент_БарКод'))

# Список числовых столбцов для суммирования
sum_cols = ['Продажи, шт.', 'Продажи в руб.', 'Себестоимость в руб.']

# 
df_for_work = df_for_work.groupby(['Клиент_БарКод', 'Номенклатура (вх)'], as_index=False).agg({
    **{col: 'sum' for col in sum_cols},  # Числовые столбцы — сумма
    **{col: lambda x: ', '.join(map(str, x.dropna().unique()))  # Преобразуем в строки и удаляем NaN
       for col in df_for_work.columns if col not in sum_cols 
       and col not in ['Клиент_БарКод', 'Номенклатура (вх)']}
})

# Пересобираем DataFrame после обработки
df_for_work = df_for_work[['Клиент_БарКод', 'Номенклатура (вх)', 'Продажи, шт.', 'Продажи в руб.',
       'Себестоимость в руб.', 'Клиент (вх)', 'Бар-код', 'Подгруппа товара (вх)', 
       'Поставщик РЦ', 'Тип поставщика РЦ', 'Производитель']]

## 1. Вытащим объем из наименования номенклатуры

In [ ]:
def extract_volume_advanced(text):
    if pd.isna(text):
        return None
    
    text = str(text).lower()
    
    # Шаблоны для поиска:
    patterns = [
        # Варианты с литрами (1.5л, 1,5л, 1.5 л, 1л, 1.5l)
        r'(?P<volume>\d+[,.]\d+)\s*[лl]\b',  # 1.5л или 1.5l
        r'(?P<volume>\d+)\s*[лl]\b',         # 5л или 5l
        
        # Варианты с миллилитрами (500мл, 500 мл, 500ml)
        r'(?P<volume>\d+)\s*мл\b',        # 500мл
        r'(?P<volume>\d+)\s*ml\b',        # 500ml
        
        # Просто числа перед L или [ (1.5l, 0.5l)
        r'(?P<volume>\d+[,.]\d+)\s*(?=[\[l])',     # 1.5 перед [ или l
        r'(?P<volume>\d+)\s*(?=[\[l])',            # 5 перед [ или l
        
        # Варианты с указанием "литр" (1.5 литра)
        r'(?P<volume>\d+[,.]\d+)\s*литр\w*',
        r'(?P<volume>\d+)\s*литр\w*',
        
        # Шаблоны для случаев без единиц измерения
        r'(?P<volume>\d+[,.]\d+)\s*(?=\s*б/г)',    # 1.5 перед "б/г"
        r'(?P<volume>\d+)\s*(?=\s*б/г)',           # 5 перед "б/г"
        r'(?P<volume>\d+[,.]\d+)\s*(?=\s*пэт)',    # перед "пэт"
        r'(?P<volume>\d+)\s*(?=\s*пэт)',
        r'(?P<volume>\d+[,.]\d+)\s*(?=\s*негаз)',  # перед "негаз"
        r'(?P<volume>\d+)\s*(?=\s*негаз)',
        r'(?P<volume>\d+[,.]\d+)\s*$',             # число в конце строки
        r'(?P<volume>\d+)\s*$',
        r'\b(?P<volume>\d+[,.]\d+)\b',             # любое число с запятой/точкой
        r'\b(?P<volume>\d+)\b'                     # любое целое число
    ]
    
    for pattern in patterns:
        match = re.search(pattern, text)
        if match:
            volume = match.group('volume').replace(',', '.')
            try:
                volume_float = float(volume)
                # Если был шаблон с мл - переводим в литры
                if 'мл' in pattern or 'ml' in pattern:
                    return volume_float / 1000
                # Фильтрация слишком больших чисел (вероятно, не объёмы)
                if volume_float > 30:  # предполагаем, что объёмы больше 30 литров маловероятны
                    continue
                return volume_float
            except ValueError:
                continue
                
    return None

# Запуск функции
df_for_work['Объем_л'] = df_for_work['Номенклатура (вх)'].apply(extract_volume_advanced)

# Проверка возможных ошибок и вывод их в удобном формате
df_log_volume = df_for_work[df_for_work['Объем_л'].isnull()]
print(df_log_volume['Номенклатура (вх)'].value_counts().to_dict())

## 2. Вытащим вид упаковки из наименования номенклатуры

In [ ]:
def extract_package_enhanced(text):
    if pd.isna(text):
        return None
    
    text = str(text).upper()
    
    # Основной словарь упаковок с приоритетами
    package_rules = [
        # 1. Явные указания упаковки (высший приоритет)
        (r'(ПЭТ|СТЕКЛО|Ж/Б|ТЕТРАПАК|ФЛЕКСОПАК)[^А-Я]', lambda m: m.group(1).lower()),
        
        # 2. Специальные правила для стекла (высокий приоритет)
        (r'СТ\.?\s*(?=[\(\[]|$)', 'стекло'),  # "ст." или "ст" перед скобкой или концом строки
        (r'\.СТ\.?\s*(?=[\(\[]|$)', 'стекло'),  # ".СТ." или ".СТ" перед скобкой
        (r'[^А-Я]СТ\.?\s*(?=[\(\[]|$)', 'стекло'),  # " СТ." или " СТ" перед скобкой
        
        # 3. Стандартные сокращения (средний приоритет)
        (r'\bСТ[/ ]?Б\b', 'стекло'),
        (r'\bСТ[/ ]?БУТ\b', 'стекло'),
        (r'\bС[/ ]?Б\b', 'стекло'),
        (r'\bПЭТ\b', 'пэт'),
        (r'\bПЛ[/ ]?Б\b', 'пэт'),
        (r'\bПЛАСТ[/ ]?БУТ\b', 'пэт'),
        (r'\bЖ[/ ]?Б\b', 'ж/б'),
        (r'\bАЛ[/ ]?Б\b', 'ж/б'),
        
        # 4. Контекстные правила (низкий приоритет)
        (r'БУТЫЛК', 'стекло'),
        (r'СТЕКЛ', 'стекло'),
        (r'ПЭТ[^А-Я]', 'пэт'),
    ]
    
    # Проверка правил по порядку
    for pattern, package in package_rules:
        if isinstance(package, str):
            if re.search(pattern, text):
                return package
        else:  # для callable-правил
            match = re.search(pattern, text)
            if match:
                return package(match)
    
    return None

# Запуск функции
df_for_work['Упаковка1'] = df_for_work['Номенклатура (вх)'].apply(extract_package_enhanced)

# Проверка возможных ошибок и вывод их в удобном формате
df_log_pack = df_for_work[df_for_work['Упаковка1'].isnull()]
print(df_log_pack['Номенклатура (вх)'].value_counts())

## 3. Вытащим вид газации из наименования номенклатуры

In [ ]:
def add_gazation_column(df):

    # Паттерны для газированной воды (регистронезависимо)
    gaz_patterns = [
        r'газ', r'с/г', r'газир', r'газации', r'с газом', 
        r'carbonated', r'газ\.', r'ср/г', r'с газ'
    ]
    
    # Паттерны для негазированной воды
    neg_patterns = [
        r'негаз', r'н/г', r'б/г', r'без газа', 
        r'non.?gas', r'still', r'н\.г', r'не газ'
    ]
    
    def check_gazation(text):
        if pd.isna(text):
            return None
        
        text = str(text).lower()
        has_gaz = any(re.search(pattern, text) for pattern in gaz_patterns)
        has_neg = any(re.search(pattern, text) for pattern in neg_patterns)
        
        if has_neg:
            return 'негаз'
        elif has_gaz:
            return 'газ'
        else:
            return None  # Пусто, если информация отсутствует
    
    df['Газация'] = df['Номенклатура (вх)'].apply(check_gazation)
    return df

# Запуск функции
df_for_work = add_gazation_column(df_for_work)

# Проверка возможных ошибок и вывод их в удобном формате
df_log_gazation = df_for_work[df_for_work['Газация'].isnull()]
print(df_log_gazation['Номенклатура (вх)'].value_counts())

## 4. Вытащим наименование бренда из наименования номенклатуры

In [ ]:
brands_mapping = {"S.Pellegrino": "S.Pellegrino", "Sanpellegrino": "S.Pellegrino", "Просто Азбука": "Просто Азбука", "Ватерлок": "Просто Азбука",  
    "Боржоми": "Боржоми", "IDS Borjomi Georgia": "Боржоми", "Черноголовка": "Черноголовка", "Архыз": "Архыз", "Архыз Vita": "Архыз", "Ессентуки": "Ессентуки",
    "Evian": "Evian", "Perrier": "Perrier", "Nestle Pure Life": "Nestle", "Vittel": "Vittel", "Volvic": "Volvic", "Sant Aniol": "Sant Aniol",
    "San Benedetto": "San Benedetto", "San Bernardo": "San Bernardo", "Fiji": "Fiji", "Contrex": "Contrex", "ФрутоНяня": "ФрутоНяня",
    "Агуша": "Агуша", "Стэлмас": "Стэлмас", "BAIKAL430": "Baikal", "Baikal Pearl": "Baikal", "LEGEND of BAIKAL": "Baikal", "Voss": "Voss",
    "Aqua Minerale": "Aqua Minerale", "Bona Aqua": "Bona Aqua", "Калинов Родничок": "Калинов Родничок", "Сенежская": "Сенежская", 
    'MEVER':'MEVER', 'Джермук':'Джермук', 'Рычал-Су':'Рычал-Су', 'Donat Mg':'Donat Mg', 'Tassay':'Tassay', 'Fleur Alpine':'Fleur Alpine',
    'Пилигрим':'Пилигрим', 'Jevea':'Jevea', 'Эльбрус':'Эльбрус', 'Нарзан':'Нарзан', "Sant'Anna":"Sant'Anna", 'Кисловодская':'Кисловодская',
    'Черноголовская':'Черноголовка', 'Набеглави':'Набеглави', 'iki':'iki', 'Prolom voda':'Prolom voda', 'Petroglyph':'Petroglyph',
    'Калинов Родник':'Калинов Родник', 'Magnesia Mattoni':'Magnesia Mattoni', 'Bilinska Kyselka':'Bilinska Kyselka', 'Fahrenheit':'Fahrenheit',
    'ICE BRICKS':'ICE BRICKS', 'Solan de cabras':'Solan de cabras', 'Baikal Reserve':'Baikal Reserve', 'Fiuggi':'Fiuggi', 'Edis':'Edis',
    'COSMOS':'COSMOS', 'NO STRESS':'NO STRESS', 'Nedra':'Nedra', 'Vincentka':'Vincentka', 'ТБАУ':'ТБАУ', 'СИЛА ВОДЫ':'СИЛА ВОДЫ', 'Мика-Мика':'Мика-Мика',
    'ACTIPH':'ACTIPH', 'GORJI':'GORJI', 'Кисловодский источник':'Кисловодский источник', "Nature's own factory":"Nature's own factory",
    'Д.Р. ЦЕЛЕБНОВ':'Д.Р. ЦЕЛЕБНОВ', 'Cristaline':'Cristaline', 'Dausuz':'Dausuz', 'CERELIA':'CERELIA', 'Mivela':'Mivela', 'Аш-тау':'Аш-тау',
    'OXY BALANCE':'OXY BALANCE', 'СанАторио':'СанАторио', 'Новотерская целебная':'Новотерская целебная', 'Vichy Celestins':'Vichy Celestins',
    'Берегиня':'Берегиня', 'Vitaoxy':'Vitaoxy', 'ARTE':'ARTE', 'Королевская':'Королевская', 'Аква Минерале':'Аква Минерале', "Святой источник": "Святой источник",
    'Заповедник Здоровья':'Заповедник Здоровья', 'Липецкий Бювет':'Липецкий Бювет', 'Стелмас Mg+':'Стелмас Mg+', 'Байкал':'Байкал', 'БонаАква':'БонаАква',
    'Горджи':'Горджи', 'Донат':'Донат', 'Тассай':'Тассай', 'Кармадон':'Кармадон', 'Волжанка':'Волжанка', 'Дивеевская':'Дивеевская',
    'Сан Бенедетто':'Сан Бенедетто', 'Эдельвейс':'Эдельвейс', 'ОКЕЙ':'ОКЕЙ', 'Стелмас':'Стелмас', 'Липецкая Росинка':'Липецкая Росинка',
    'Краснозатонская Серебряная':'Краснозатонская Серебряная', 'Нестле':'Нестле', 'Аштау':'Аштау', 'Мтаби':'Мтаби', 'Шишкин Лес':'Шишкин Лес',
    'BAIKOLOGY':'BAIKOLOGY', 'Горная Ривьера':'Горная Ривьера', 'Дупленская':'Дупленская', '4 Воды':'4 Воды', 'Славянская Жемчужина':'Славянская Жемчужина',
    'Ачалуки':'Ачалуки', 'Иркутская':'Иркутская', 'Курортная Мальтинская':'Курортная Мальтинская', 'Мивела Mg+':'Мивела Mg+',
    'Фаренгейт':'Фаренгейт', 'Сулинка Кремниевая':'Сулинка Кремниевая', 'Вкус Здоровья':'Вкус Здоровья', 'Acqua Panna':'Acqua Panna',
    'Нагутская-26':'Нагутская-26', 'БиоВита':'БиоВита', 'Мевер':'Мевер', "О'КЕЙ DAILY":"О'КЕЙ DAILY", 'Лель':'Лель', 'Живея':'Живея',
    'Окси Баланс':'Окси Баланс', 'Перье':'Перье', 'Серебряная':'Серебряная', 'Сибирский Бор':'Сибирский Бор', 'Карачинская':'Карачинская',
    'Обуховская':'Обуховская', 'Благая':'Благая', 'БонАква':'БонАква', 'Князь Милош':'Князь Милош', 'Только Вода':'Только Вода',
    'Липецкая':'Липецкая', 'Ваша Вода':'Ваша Вода', 'Аква Фит':'Аква Фит', 'Сарова':'Сарова', 'Красный Ключ':'Красный Ключ', 'Демидовъ':'Демидовъ',
    'Аквин':'Аквин', 'Долина родников':'Долина родников', 'Водопад':'Водопад', 'Тараскульская':'Тараскульская', 'Красноусольская':'Красноусольская',
    'Тюменская':'Тюменская', 'Аксинья':'Аксинья', 'Хан-Куль':'Хан-Куль', 'Арктик':'Арктик', 'Светла':'Светла', 'Виста':'Виста', 'Шадринская':'Шадринская',
    'Кристэль':'Кристэль', 'Хилак':'Хилак', 'Пролом':'Пролом', 'Арже':'Арже', 'Омская':'Омская', 'Суздальские напитки':'Суздальские напитки',
    'Каролина Минерале':'Каролина Минерале', 'Люкс':'Люкс', 'Суздальские напитки':'Суздальские напитки', 'Раифский источник':'Раифский источник', 
    'Хрустальный колодец':'Хрустальный колодец', 'Arctiс':'Arctiс', 'Mguri':'Mguri', 'Старый источник':'Старый источник', 'Aqua Tonus':'Aqua Tonus',
    'Живой Ключ':'Живой Ключ', 'ГОРНЫЙ ИСТОЧНИК':'ГОРНЫЙ ИСТОЧНИК', 'ЛЕГЕНДА ЖИЗНИ':'ЛЕГЕНДА ЖИЗНИ', 'ЛЕГЕНДА ЖИЗ':'ЛЕГЕНДА ЖИЗНИ', 'ШИФАЛЫ СУ':'ШИФАЛЫ СУ',
    'АЛТАЙСКИЙ РОДНИК':'АЛТАЙСКИЙ РОДНИК', 'СВ. ИСТОЧ.':'СВЯТОЙ ИСТОЧНИК', 'СВ.ИСТОЧ.':'СВЯТОЙ ИСТОЧНИК', 'КИСЛОВОДСКИЙ ИСТ.':'КИСЛОВОДСКИЙ ИСТОЧНИК',
    'LEG OF BAIKAL':'Байкал','LEG.OF BAIKAL':'Байкал', 'КРЫМ':'КРЫМ', 'АКВАБАЛАНС':'АКВАБАЛАНС', 'КД/':'Каждый день', 'SAN PELLEGR':'S.Pellegrino',
    'ЗАПОВЕДНЫЙ ИСТ.':'ЗАПОВЕДНЫЙ ИСТОЧНИК', 'СЕРАФИМОВ ДАР':'СЕРАФИМОВ ДАР', 'ЭЛИТА':'ЭЛИТА', 'ШИФАСУ':'ШИФАСУ', 'ЧИСТОГОРЬЕ':'ЧИСТОГОРЬЕ',
    'ST ANIOL':'ST ANIOL', 'ЧИСТ ГЛ':'Чистый глоток', 'ТАРАСКУЛЬ.':'ТАРАСКУЛЬСКАЯ', 'ГОР.КЛЮЧ2000':'ГОРЯЧИЙ КЛЮЧ', 'ХАН КУЛЬ':'ХАН КУЛЬ',
    'КАПЕЛЬ':'КАПЕЛЬ', 'АЛТАЙСКИЙ ИСТ':'АЛТАЙСКИЙ ИСТОЧНИК', 'РАМЕНО':'РАМЕНО', 'ЧИСТ.ГЛОТОК':'Чистый глоток', 'MAGNETISTA':'MAGNETISTA',
    'КРАСНОУС.':'КРАСНОУСОЛЬСКАЯ', 'КАЛИНОВ':'Калинов родник', 'ЛЕГ СИБИРИ':'ЛЕГЕНДА СИБИРИ', 'ЛЕГ СИБ':'ЛЕГЕНДА СИБИРИ', 'КУБАЙ':'КУБАЙ',
    'КП/ВОДА':'Красная птица', 'СЕНЕЖ.':'СЕНЕЖСКАЯ', 'ВОДА НК':'Ново-Курьинская', 'СВЕТЛРОДНИК':'СВЕТЛЫЙ РОДНИК', 'СВЕТЛЫЙ РОДНИК':'СВЕТЛЫЙ РОДНИК',
    'АКВА-ВАЙТ':'Бештау', 'A-PREMIUM':'A-PREMIUM', 'БЕЛЫЙ КЛЮЧ':'БЕЛЫЙ КЛЮЧ', 'РОДНИКИ ЛАГОНАКИ':'РОДНИКИ ЛАГОНАКИ', 'СЕРАФИМОВДАР':'СЕРАФИМОВДАР',
    'AQUAMIX':'AQUAMIX', 'НОВОКУМ.':'НОВОКУМСКАЯ', 'ОБУХОВСК.':'ОБУХОВСКАЯ', 'ОБУХ.':'ОБУХОВСКАЯ', 'ЭВЕРЕСТ':'ЭВЕРЕСТ',
    'ЛАГУНА':'ЛАГУНА', 'СИБИР.БОР':'СИБИРСКИЙ БОР', 'УВИН ЖЕМЧУЖ':'Увинская Жемчужина', 'СИБ БОР':'СИБИРСКИЙ БОР', 'MISS MINERAL':'MISS MINERAL',
    'ВУНДЕРВЕЛЛЕ':'ВУНДЕРВЕЛЛЕ', 'АССОЛЬ':'АССОЛЬ', 'КАЗБЕК-АКВА':'КАЗБЕК-АКВА', 'АМ АКТИВ':'АКВА МИНЕРАЛЕ', 'РОДНИКОФФ':'РОДНИКОФФ',
    'ГОР КЛЮЧ':'Горячий ключ', 'СИЛА ЗЕМЛИ':'СИЛА ЗЕМЛИ', 'LIFELINE':'LIFELINE', 'КАП-ЛИК':'КАП-ЛИК', 'СВ.ИСТОЧНИК':'Святой источник',
    'СВ. ИСТОЧНИК':'Святой источник', 'NIAGARA':'NIAGARA', 'ДУПЛЕНС':'ДУПЛЕНС', 'БЕЛЫЕ ГОРЫ':'БЕЛЫЕ ГОРЫ', 'ПИТ.URAL':'URAL',
    'МЕДВЕЖИЙ КЛЮЧ':'МЕДВЕЖИЙ КЛЮЧ', 'ВЛАСОВ КЛЮЧ':'ВЛАСОВ КЛЮЧ', 'ВЛАС КЛ':'ВЛАСОВ КЛЮЧ', 'АЛТА АКВА':'АЛТА АКВА', 'РОДНОЙ БЮВЕТЪ':'РОДНОЙ БЮВЕТЪ',
    'СЕСТРИЦА':'СЕСТРИЦА', 'ХРУСТАЛ КОЛОДЕЦ':'ХРУСТАЛЬНЫЙ КОЛОДЕЦ', 'ХРУСТАЛ КОЛОД':'ХРУСТАЛЬНЫЙ КОЛОДЕЦ', '7 РУЧЬЕВ':'7 РУЧЬЕВ',
    'КРИСТАЛЛИНКА':'КРИСТАЛЛИНКА', 'АКВА-СПЕЙС':'АКВА-СПЕЙС', 'ЗЮРАТКУЛЬ':'ЗЮРАТКУЛЬ', 'КРАСН.КЛЮЧ':'КРАСНЫЙ КЛЮЧ',
    'РОДНИК БЕЛОГОР.':'РОДНИК БЕЛОГОРЬЯ', 'ЧЕБАРКУЛЬ':'ЧЕБАРКУЛЬ', 'ОШТЕН':'ОШТЕН', 'AQUAFIT':'AQUAFIT', 'NESTLE':'NESTLE',
    'БУКЕТ ЧУВАШИИ':'БУКЕТ ЧУВАШИИ', 'АКВА АКТИВ':'АКВА АКТИВ', 'AQUAФИШТ':'AQUAФИШТ', 'ЛЕГЕН ЖИЗНИ':'ЛЕГЕНДА ЖИЗНИ', 'ВОДА ДОНА':'ВОДА ДОНА',
    'ВОЛЖАН':'ВОЛЖАНКА', 'BORJOMI':'BORJOMI', 'КАЛИН.РОДНИК':'КАЛИНОВ РОДНИК', 'БАГИАТТИ':'БАГИАТТИ', 'Св.ист.':'Святой источник',
    'КАЛИН.РОД.':'КАЛИНОВ РОДНИК', 'SVETLA':'SVETLA', 'WATERFUL':'WATERFUL', 'ЖЕМ.БАЙК.25МBAIKALSEA':'Байкал', 'BAIK.WATER':'Байкал',
    'ЧЕРНОГ.':'ЧЕРНОГОЛОВКА', 'JEYEA CRYSTAL':'JEYEA', 'РОДНИКИ САИРМЕ':'РОДНИКИ САИРМЕ', 'S.PELLEG':"S.Pellegrino", 'AQUA MIN':'Аква минерале',
    'BAIK.PEARL':'Байкал', 'ВАШ ВЫБОР':'ВАШ ВЫБОР', 'Д.Р.ЦЕЛЕБНОВ':'Д.Р.ЦЕЛЕБНОВ', 'HIL-AQUA':'HIL-AQUA', 'KNJAZ MILOS':'KNJAZ MILOS',
    'САН БЕРНАРДО':'САН БЕРНАРДО', 'НАПИТОК ГЛОБУС':'ГЛОБУС', 'ЛИП.РОСИНКА':'ЛИПЕЦКАЯ РОСИНКА', 'BILIN.KYS.':'Bilinska Kyselka', 'ЛЕГ.СИБ.':'ЛЕГЕНДА СИБИРИ',
    'АРЗА':'АРЗА', 'NATURELIA':'NATURELIA', 'RARE':'RARE', 'КИСЛОВ.ЦЕЛ.':'КИСЛОВОДСКАЯ ЦЕЛЕБНАЯ', 'ORION':'ORION', 'АДЫЛ ТАУ':'АДЫЛ ТАУ', 
    'APARAN':'APARAN', 'ПИЛИГ.':'ПИЛИГРИМ', 'ШИШК.ЛЕС':'ШИШКИН ЛЕС', 'ПСЫЖ':'ПСЫЖ', 'СЕМЬ РУЧЬЕВ':'СЕМЬ РУЧЬЕВ', 'МЕРКУРИЙ':'МЕРКУРИЙ',
    'BAIK.RES.':'Байкал', 'КИСЛОВОД.ЦЕЛЕБ':'КИСЛОВОДСКАЯ ЦЕЛЕБНАЯ', 'КР.ЦЕНА':'КРАСНАЯ ЦЕНА', 'АКВАТОНУС':'АКВАТОНУС', 'FIZZBERRY':'FIZZBERRY',
    'МАРКЕТ Вода':'МАРКЕТ', 'MIZU':'MIZU', 'ЗАПОВ.ЗДОР.':'ЗАПОВЕДНИК ЗДОРОВЬЯ', 'MIV.MG':'Mivela Mg', 'РОС.Вода ЛИП.':'ЛИПЕЦКАЯ', 'ACQ.PAN.':'Acqua Panna',
    'НАБЕГЛ.':'Набеглави', 'ЛИПЕЦК.':'ЛИПЕЦКАЯ', 'БОРЖ.':'Боржоми', 'ЛИП.РОСИН.':'ЛИПЕЦКАЯ РОСИНКА', 'МИВЕЛА MG':'Mivela Mg', 'HANTICA':'HANTICA',
    'СЕРНОВОД.':'СЕРНОВОДСКАЯ', 'LEG.BAIK.':'Байкал', 'РОСИН.Нап.ЛИП.ЛАЙТ':'ЛИПЕЦКАЯ РОСИНКА', 'СЕРЕБ.УСТЬ-БЫС.':'СЕРЕБРЯНАЯ', 'МЕД.КЛЮЧ':"МЕДВЕЖИЙ КЛЮЧ", 
    'ТЕБЕРЕЛЬ':'ТЕБЕРЕЛЬ', 'ШИШ.ЛЕС':'ШИШКИН ЛЕС', 'ЗАВ.РОДНИК':'ЗАВЕТНЫЙ РОДНИК', 'АЛЛ.ИСТ.':'АЛЛЕЯ ИСТОЧНИКОВ', 'НИАГАРА':'НИАГАРА',
    'ЗАПОВЕДНИК ЗДОРОВ':'ЗАПОВЕДНИК ЗДОРОВЬЯ', 'ИСТОЧНИК СТАРО-МЫТИЩИНСКИЙ':'ИСТОЧНИК СТАРО-МЫТИЩИНСКИЙ', 'ДОНА':'ДОНА', 'AXOUT':'AXOUT',
    'КРИСТАЛЬНЫЙ РОДНИК':'КРИСТАЛЬНЫЙ РОДНИК', 'СУЗД.НАПИТКИ':'СУЗДАЛЬСКИЕ НАПИТКИ', 'УЗМЕНЬ':'УЗМЕНЬ', 'МЕНЗЕЛИН.':'МЕНЗЕЛИНСКАЯ',
    'М.СТИХ.':'МОЯ СТИХИЯ', 'СВЕЖЕСТЬ':'СВЕЖЕСТЬ', 'ГОР.ВЕРШИНА':'ГОРНАЯ ВЕРШИНА', 'ГОР.ВЕРШ.':'ГОРНАЯ ВЕРШИНА', 'ХАНТИКА':'ХАНТИКА',
    'ЛЕГЕНДА СИБИРИ':'ЛЕГЕНДА СИБИРИ', 'САИРМЕ':'САИРМЕ', 'SANT AN.':'Sant Aniol', 'ДОБРЫЙ':'ДОБРЫЙ', 'ЛИП.БЮВЕТ':'ЛИПЕЦКИЙ БЮВЕТ', 
    'SOUROTI':'SOUROTI', 'РАИФСК.ИСТОЧ.':'РАИФСКИЙ ИСТОЧНИК', 'Мелекесский источник':'Мелекесский источник', 'ГРАФ СУВОРОВ':'ГРАФ СУВОРОВ',
    'РУШАНОЧКА':'РУШАНОЧКА', 'МАРК.ПЕРЕКР.':'МАРКЕТ ПЕРЕКРЕСТОК', 'СИНЕБОРЬЕ':'СИНЕБОРЬЕ', 'КУВАКА':'КУВАКА', 'ВОРТЕКС':'ВОРТЕКС', 'VODA':'VODA',
    'BURYATIA':'BURYATIA', 'УСПОЛОНЬ':'УСПОЛОНЬ', 'АРТЕЗИАНСКИЙ ИСТОК':'АРТЕЗИАНСКИЙ ИСТОК', 'БАЙК.ЛЕДН.':'Байкал', 'МАЛАХОВ.':'МАЛАХОВСКАЯ',
    'ХРУСТАЛ.ИСТОК':'ХРУСТАЛЬНЫЙ ИСТОК', 'СЕСТР.-ПРИР.':'СЕСТРИЦА-ПРИРОДНАЯ', 'СВЕТ.РОДНИК':'СВЕТЛЫЙ РОДНИК', 'РОДН.Л-Н.Вода':'РОДНИКИ ЛАГО-НАКИ',
    'ЛЕГ.ГОР.АРХ.':'АРХЫЗ', 'ЗАПОВ.ИСТ.':'ЗАПОВЕДНЫЙ ИСТОЧНИК', 'KN.MIL.Вода':'Knjaz Milos', 'АР.МЭР.Вода':'Аргали Мэргэн', 'УВИНСК.ЖЕМЧ.Вода':'УВИНСКАЯ ЖЕМЧУЖИНА',
    'ГОР.КЛ.2-2000':'Горячий ключ', 'S.PEL.Вода':'S.Pellegrino', 'ДОН.Вода':'ДОНА', 'АЛАН.ГОР.Вода':'Аланийская горная', 'ОБУХОВС.13':'Обуховская', 
    'ZAJ.HORKA':'Zajecicka Horka', 'Черног':'Черноголовка', 'РОД.Л-Н.Вода':'РОДНИКИ ЛАГО-НАКИ', 'СИБИРС.БОР':'СИБИРСКИЙ БОР', 'ЭРИДАН':'ЭРИДАН',
    'ИВОЛГА':'ИВОЛГА', 'MONTCLAR':'MONTCLAR', 'BASHLAM':'BASHLAM', 'VARIA':'VARIA', 'ЦЕЛЕБ.ИСТОК':'ЦЕЛЕБНЫЙ ИСТОК', 'КАРЕЛ.ЖЕМЧ.Вода':'КАРЕЛЬСКАЯ ЖЕМЧУЖИНА', 
    'СЕРЕБ.УСТЬ-БЫСТ.':'СЕРЕБРЯНАЯ', 'КР.КЛЮЧ':'КРАСНЫЙ КЛЮЧ', 'РОДН.БЕЛОГОР.':'РОДНИКИ БЕЛОГОРЬЯ', 'МЕНЗЕЛ.':'МЕНЗЕЛИНКА', 'СИБ.БОР':'СИБИРСКИЙ БОР',
    'БУК.ЧУВ.Напиток':'БУКЕТ ЧУВАШИИ', 'МЕНЗ.':'МЕНЗЕЛИНКА', 'УВИНСК.ЛУЛВУ':'УВИНСКАЯ ЛУЛВУ', 'МАРК.ПЕР.Вода':'МАРКЕТ ПЕРЕКРЕСТОК', 'ЗЕЛ.ЛИН.Вода':'ЗЕЛЕНАЯ ЛИНИЯ',
    'ДО ВСТРЕЧИ':'ДО ВСТРЕЧИ', 'LEADER':'LEADER', 'РЕЦ.ОТ ПР.Вода':'РЕЦЕПТ ОТ ПРИРОДЫ', 'САРАНА':'САРАНА', 'ГОР.КЛЮЧ':'ГОРЯЧИЙ КЛЮЧ', 'ФЛЮОРИ':'ФЛЮОРИ',
    'BURABAY':'BURABAY', 'NORDPILGRIM':'NORDPILGRIM', 'СЕРЕБ.РОСА':'СЕРЕБРЯНАЯ РОСА', 'ИСТОК':'ИСТОК', 'РЕЧИСТА':'РЕЧИСТА', 'ОШМЕС':'ОШМЕС',
    'SOCIETE MIN.Вода':'Societe Minerale', 'КАРАЧИН.Вода':'Карачинская', 'SOC.MINERALE':'Societe Minerale', 'SOC.MIN.Вода':'Societe Minerale',
    'КРАСНОЗАТОН.СЕРЕБР.':'КРАСНОЗАТОНСКАЯ СЕРЕБРЯНАЯ', 'АЛТАЙ АКВА':'АЛТАЙ АКВА', 'ГОР.ИСТ.Вода':'ГОРНЫЙ ИСТОЧНИК', 'СЕРЕБ.ДАР':'СЕРЕБРЯНЫЙ ДАР',
    'УВИН.ЖЕМЧ.Вода':'УВИНСКАЯ ЖЕМЧУЖИНА', 'ЗДОР.ВОДА':'ЗДОРОВАЯ ВОДА', 'КАР.PREM.Вода':'Карельская Premium', 'ПР!СТ':'ПЕРЕКРЕСТОК', 
    'ГОС.В.НОВ.Вода':'Новотерская', 'S.ANNA':'SANT_ANNA', 'D.BORM.Вода':'Доктор Броменталь', 'ВОЛЖ.ДАР':'ВОЛЖСКИЙ ДАР', 'FERD.PRAM.Вода':'Ferdinanduv pramen',
    'ЧИСТОЗЕР.Вода':'ЧИСТОЗЕРНАЯ', 'ARIV.Вода':'Arivali', 'Р.ПРИКАМ.Вода':'РОДНИК ПРИКАМЬЯ', 'НОВОКУРЬИН.Вода':'НОВОКУРЬИНСКАЯ',
    'КРАСНОУСОЛ.РОДНИК':'КРАСНОУСОЛЬСКАЯ', 'CRANZWASSER':'CRANZWASSER', 'МАЙКОП.Вода':'МАЙКОПСКАЯ', 'ЗЕЛЕН.НАСТ.':'ЗЕЛЕНОГРАДСКАЯ', 'ДАРИДА':'ДАРИДА',
    'ЗЕЛЕНОГР.СКВАЖ.624Д':'ЗЕЛЕНОГРАДСКАЯ', 'ЗЕЛ.СКВ.624Д':'ЗЕЛЕНОГРАДСКАЯ', 'ОМСК.№1':'ОМСКАЯ', 'ИОНИКА':'ИОНИКА', 'БИШУЛИ':'БИШУЛИ',
    'SOC.MIN.Нап.':'Societe Minerale', 'ИРКУТ.ПРИР.':'ИРКУТСКАЯ ПРИРОДНАЯ', 'ТЕРЕК':'ТЕРЕК', 'ШХЕЛЬДА':'ШХЕЛЬДА', 'ГОР.ИСТОЧ.пит.':'ГОРНЫЙ ИСТОЧНИК', 
    'КРАСНОЗАТОН.СЕРЕБ.':'КРАСНОЗАТОНСКАЯ', 'ЗЕЛЕНОГР.НАСТ.':'ЗЕЛЕНОГРАДСКАЯ', 'БЕЛОК.ВОС.№2 ':'Белокурихинская', 'ЗЮРАТ.АРДВИ':'Зюраткуль ардви',
    'КЛАДОВ.МИНЕР.':'Кладовая минералов', 'КЛ.МИН.':'Кладовая минералов', 'ЗЮРАТ.АРДВИ':'Зюраткуль ардви', 'СЛАВДА':'СЛАВДА', 'ЛЕГЕН.ЖИЗ.Вода':'ЛЕГЕНДА ЖИЗНИ',
    'PRIMAVERA':'PRIMAVERA', 'ЧУДСК.РОСА':'ЧУДСКАЯ РОСА', 'ШМАКОВКА':'ШМАКОВКА', 'КРАНЦВАССЕР':'КРАНЦВАССЕР', 'ЛОТОС':'ЛОТОС', 'ROKADA':'ROKADA',
    'АЛТАЙ':'АЛТАЙ', 'КРАСНОУСОЛ.ЦЕЛ.':'Красноусольская Целебная', 'РОД.ПРИКАМ.':'РОДНИК ПРИКАМЬЯ', 'СВЕТ.РОДН.Вода':'СВЕТЛЫЙ РОДНИК', 
    'СЕР.ИСТ.Вода':'СЕРЕБРЯНЫЙ ИСТОЧНИК', 'СКИТ Вода МЕДВЕЖ.':'Медвежка', 'АКВА ВИВА':'АКВА ВИВА', 'ДОЛ.РОДН.Вода':'ДОЛИНА РОДНИКОВ', 
    'СКИТ Вода ШМАК.1':'ШМАКОВКА', 'ЗАП.ЗД.Вода':'ЗАПОВЕДНИК ЗДОРОВЬЯ', 'БАЛТ.ПАРУС':'БАЛТИЙСКИЙ ПАРУС', 'КУКА-7':'КУКА', 'МИНЕРАЛИКА':'МИНЕРАЛИКА',
    'РОМАНОВ.Вода':'РОМАНОВСКАЯ', 'КУКА':'КУКА', 'CISOWIANKA':'CISOWIANKA', 'ЧИСТОЗЕРЬЕ':'ЧИСТОЗЕРЬЕ', 'ЖИВ.ВОДА':'ЖИВАЯ ВОДА', 'НОРМА ЖИЗНИ':'НОРМА ЖИЗНИ',
    'ПРИМАВЕРА':'ПРИМАВЕРА', 'СОСНОВ.РОСА':'СОСНОВСКАЯ РОСА', 'ЧУДОТВ.Вода':'ЧУДОТВОРНАЯ', 'СУЛИНКА':'СУЛИНКА', 'ТЕРСИН.Вода':'ТЕРСИНКА',
    'BON-AQUA':'BONAQUA', 'ЗАВЕТНЫЙ РОДНИК':'ЗАВЕТНЫЙ РОДНИК', 'МАГНИТ':'МАГНИТ', 'AQUANIKA':'AQUANIKA', 'ТАРАСКУЛ.Вода':'ТАРАСКУЛЬСКАЯ',
    'УЛАН-УДЭН.Вода':'Улан-Удэнская', 'Суздал напит':'Суздальские напитки', 'СВЕЖ АЛТ':'СВЕЖЕСТЬ АЛТАЯ', 'КРИСТАЛ.':'КРИСТАЛЬНАЯ', 'ТРОНОША':'ТРОНОША',
    'ЛИП.БЮВ.':'ЛИПЕЦКИЙ БЮВЕТ', 'THONON':'THONON', 'КЛЮЧИ':'КЛЮЧИ', 'Эвиан':'Эвиан', 'Зеленый Городок':'Зеленый Городок','СВЯТ.ИСТ.':'Святой источник',
    'Моя Стихия':'Моя Стихия', 'Цхали':'Цхали', 'Журавлево':'Журавлево', 'Тарханская':'Тарханская', 'КМВ ОЛОНИЯ':'Олония', 'СОБОВСКАЯ':'СОБОВСКАЯ', 
    'СОДОВАЯ':'СОДОВАЯ', 'Shifa su':'Shifa su', 'СЕНЕЖС.':'СЕНЕЖСКАЯ', 'КАЛ.РОД.Вода':'Калинов родник', 'УГЛИЧСКАЯ':'УГЛИЧСКАЯ', 'СВЯТ.ИСТОЧН.':'Святой источник',
    'ARCTIC':'ARCTIC', 'ВИТТЕЛЬ':'ВИТТЕЛЬ', 'ИВКИНСКАЯ':'ИВКИНСКАЯ', 'МАЙКОПСКАЯ':'МАЙКОПСКАЯ', 'Три Пятёрки':'Три Пятёрки', 'Журавлёво':'Журавлево', 
    'КАШИНСКАЯ':'КАШИНСКАЯ', 'СЕРНОВОДСКАЯ':'СЕРНОВОДСКАЯ', 'СЛАВЯНОВ.Вода':'СЛАВЯНОВСКАЯ', 'МАЙСКАЯ ХРУСТ.':'МАЙСКАЯ ХРУСТАЛЬНАЯ', 'СЕБРЯКОВСКАЯ':'СЕБРЯКОВСКАЯ',
    'НАГУТСКАЯ':'НАГУТСКАЯ', 'СЛАВЯНОВСКАЯ':'СЛАВЯНОВСКАЯ', 'УЛЬЯНКА':'УЛЬЯНКА', 'УЛЕЙМСКАЯ':'УЛЕЙМСКАЯ', 'ПИКОН':'ПИКОН', 'HEY!':'HEY!', 'БУЯН':'БУЯН',
    'ЗАПОВЕДНАЯ ВОДА':'ЗАПОВЕДНАЯ ВОДА', 'СЕБРЯКОВС.':'СЕБРЯКОВСКАЯ', 'ВУД АВА':'ВУД АВА', 'ГОРНАЯ ВЕРШИНА':'ГОРНАЯ ВЕРШИНА', 'BONVIDA':'BONVIDA', 
    'AQUALITY':'AQUALITY', 'ВОДА ПИТЬЕВАЯ ДИО':'ДИО', 'КАРЕЛЬСКАЯ ЖЕМЧУЖИНА':'КАРЕЛЬСКАЯ ЖЕМЧУЖИНА', '365 ДНЕЙ':'365 ДНЕЙ', 'ЖИВАЯ ВОДА':'ЖИВАЯ ВОДА',
    'МОЯ ЦЕНА':'МОЯ ЦЕНА', 'СЕБРЯКОВ':'СЕБРЯКОВСКАЯ', 'УТРЕННЯЯ ЗВЕЗДА':'УТРЕННЯЯ ЗВЕЗДА', 'РОДНИК БЕЛОГОРЬЯ':'РОДНИК БЕЛОГОРЬЯ', 'СТАРОРУССКАЯ':'СТАРОРУССКАЯ',
    'ЛЕБЕДУШКА':'ЛЕБЕДУШКА', 'МАЙСКАЯ ХРУСТАЛ.':'МАЙСКАЯ ХРУСТАЛЬНАЯ', 'ЯСН.КОЛОДЕЦ':'ЯСНЫЙ КОЛОДЕЦ', 'БОРСКАЯ':'БОРСКАЯ', 'ДВОРЦОВАЯ':'ДВОРЦОВАЯ', 
    'ЛУЧИСТАЯ':'ЛУЧИСТАЯ', 'БЕЛЕБЕЕВСКАЯ':'БЕЛЕБЕЕВСКАЯ', 'KASHINKA':'KASHINKA', 'Увинская Жемчужина':'Увинская Жемчужина', 'KASHINKA':'KASHINKA',
    'ЧИСТАЯ ВОДА 100% ':'ЧИСТАЯ ВОДА', 'КОЗЕЛЬСКАЯ':'КОЗЕЛЬСКАЯ', 'РОДНИКОВАЯ+':'РОДНИКОВАЯ', 'АЛЛЕЯ ИСТОЧНИКОВ':'АЛЛЕЯ ИСТОЧНИКОВ', 'ИНЗЕРА':'ИНЗЕРА', 
    'Вода питьевая DIO':'ДИО','Хотнинская':'Хотнинская', 'Новотерская':'Новотерская', 'УВИНСКАЯ':'УВИНСКАЯ', 'ГАЛЕРЕЯ ИСТОЧНИКОВ':'ГАЛЕРЕЯ ИСТОЧНИКОВ',
    'ГОР.КЛ.№934':'ГОРЯЧИЙ КЛЮЧ', 'Г.КЛ.С.№934':'ГОРЯЧИЙ КЛЮЧ', 'ПРОЗРАЧНАЯ':'ПРОЗРАЧНАЯ', 'LEG.OF BAIK.':'Байкал', 'СВЯТ.ИСТОЧНИК':'Святой источник',
    'ГОР.ПОЛЯНА':'ГОРНАЯ ПОЛЯНА', 'КРАСНАЯ ГОРКА':'КРАСНАЯ ГОРКА', 'КНЕЖИЦА':'КНЕЖИЦА', 'РАДАНКА':'РАДАНКА', 'АКВАЛАЙТ':'АКВАЛАЙТ', 'ЯР ЧАЛЛЫ':'ЯР ЧАЛЛЫ',
    'МАЙСКАЯ ХРУСТАЛЬНАЯ':'МАЙСКАЯ ХРУСТАЛЬНАЯ', 'ГОРНАЯ ПОЛЯНА':'ГОРНАЯ ПОЛЯНА', 'СУРГУТСКАЯ':'СУРГУТСКАЯ', 'ГОРЯЧИЙ КЛЮЧ':'ГОРЯЧИЙ КЛЮЧ',
    'КУРТЯЕВСКАЯ':'КУРТЯЕВСКАЯ', 'КРАСНОУСОЛ.Вода':'КРАСНОУСОЛЬСКАЯ', 'ИРБИС':'ИРБИС', 'ЗАТИШЕНСКАЯ':'ЗАТИШЕНСКАЯ', 'ЗАВЬЯЛОВСКАЯ':'ЗАВЬЯЛОВСКАЯ',
    'ЛЕНТА':'ЛЕНТА', 'ЮЖНОУРАЛЬСКАЯ':'ЮЖНОУРАЛЬСКАЯ', 'ПОЛЯР.ЛЕДНИК':'ПОЛЯРНЫЙ ЛЕДНИК', 'ЭЛИТНАЯ-1':'ЭЛИТНАЯ', 'POKROVSKA':'POKROVSKA',
    'ОРСКИЙ ХЛЕБ':'ОРСКИЙ ХЛЕБ', 'KOBI':'KOBI', 'БОРОВАЯ':'БОРОВАЯ', 'МИНСКАЯ':'МИНСКАЯ', 'ЯНТ.АЙСБ.':'ЯНТАРНЫЙ АЙСБЕРГ', 'АРМЕЙСКАЯ':'АРМЕЙСКАЯ',
    'ULANTA':'ULANTA', 'ПИТЬЕВАЯ 555':'555', 'РОДНИКИ ВЕРХНЕВОЛЖЬЯ':'РОДНИКИ ВЕРХНЕВОЛЖЬЯ', 'AQUA MIX':'AQUA MIX', 'АРИАНТ':'АРИАНТ',
    'СЕРЕБРЯНЫЙ ДАР':'СЕРЕБРЯНЫЙ ДАР', 'КАСМАЛИНСКАЯ':'КАСМАЛИНСКАЯ', 'CЕНЕЖСКАЯ':'CЕНЕЖСКАЯ', 'ПЕТРОГЛИФ':'ПЕТРОГЛИФ', 'ОСТРОВСКАЯ':'ОСТРОВСКАЯ',
    'ИВЕРСКАЯ':'ИВЕРСКАЯ', 'ИСТОЧНИК СТАРО-МЫТИЩ':'ИСТОЧНИК СТАРО-МЫТИЩ', 'AQUEEN':'AQUEEN', 'МОНАХОВА КРИНИЦА':'МОНАХОВА КРИНИЦА', 'СВЕТЛОЯР':'СВЕТЛОЯР',
    'ДАНАЙ':'ДАНАЙ', 'САССИ':'САССИ', 'ПИТЬЕВАЯ №1':'ПИТЬЕВАЯ №1', 'ПУХЛЯКОВ.Вода':'ПУХЛЯКОВСКАЯ', 'НОВОКУРЬИНСКАЯ':'НОВОКУРЬИНСКАЯ', 'КОРЕНСКАЯ':'КОРЕНСКАЯ', 
    'BONAAQUA':'BONAAQUA', 'КОМСОМОЛЬСКАЯ':'КОМСОМОЛЬСКАЯ', 'ВОДА"ПИТЬЕВАЯ"№1':'ПИТЬЕВАЯ №1', 'ШМАКОВСКАЯ':'ШМАКОВСКАЯ', 'ПРИРОДНАЯ':'ПРИРОДНАЯ',
    'СЛАВЯН.Вода':'СЛАВЯНОВСКАЯ', 'СЛАВЯНОВ.':'СЛАВЯНОВСКАЯ', 'КИСЛ.ЦЕЛЕБНАЯ':'КИСЛОВОДСКАЯ ЦЕЛЕБНАЯ', 'НИЖНЕ-СЕРГИНСКАЯ':'НИЖНЕ-СЕРГИНСКАЯ', 'ЛИПЕЦ-ЛАЙТ':'ЛИПЕЦКАЯ',
    'ВОДАЛИПЕЦ-ЛАЙТ':'ЛИПЕЦКАЯ', 'КАСАМАЛИНСКАЯ':'КАСАМАЛИНСКАЯ', 'РОДНИК ПРИКАМЬЯ':'РОДНИК ПРИКАМЬЯ', 'Для ляль':'Для ляль', 'Знак качества':'Знак качества', 
    'URAL':'URAL', 'РУССКАЯ АЛЯСКА':'РУССКАЯ АЛЯСКА', 'ИСТОЧНИК ЗДОРОВОЙ ЖИЗНИ':'ИСТОЧНИК ЗДОРОВОЙ ЖИЗНИ', 'МАХАЧКАЛА':'МАХАЧКАЛА', 'NORD PILGRIM':'Пилигрим',
    'КРИСТАЛЬНАЯ':'КРИСТАЛЬНАЯ', 'АЛЯСКА':'АЛЯСКА', 'ИНДИГОFLOW':'ИНДИГОFLOW', 'VULCANICA':'VULCANICA', 'РОДНИКОВАЯ +':'РОДНИКОВАЯ', 'СКАНДИНАВИЯ':'СКАНДИНАВИЯ',
    '69 PARALEL':'69 PARALEL', 'СЕЛЬТЕРСКАЯ':'СЕЛЬТЕРСКАЯ', 'CВЯТОЙ ИСТОЧНИК':'Святой источник', 'Вартемяжская':'Вартемяжская', 
    'НЕВСКИЙ КЛЮЧ':'НЕВСКИЙ КЛЮЧ', 'ЯРКИЙ ДЕНЬ':'ЯРКИЙ ДЕНЬ', 'АЛТАИКА':'АЛТАИКА', 'BISHULI':'BISHULI', 'БЖНИ':'БЖНИ', 'CLEWELL':'CLEWELL', 
    'Вода витаминизированная bp':'bp', 'ЗАУРАЛЬСКИЕ НАПИТКИ':'ЗАУРАЛЬСКИЕ НАПИТКИ', 'МАЛАХОВСКАЯ':'МАЛАХОВСКАЯ', 'НАП ГЛОБУС':'ГЛОБУС', 'AQUA BENE':'AQUA BENE',
    'ВОДА ГЛОБУС':'ГЛОБУС', 'СЛАВЯНОЧКА':'СЛАВЯНОЧКА', 'ПЕРВЫМ ДЕЛОМ':'ПЕРВЫМ ДЕЛОМ', 'РОКАДА':'РОКАДА', 'СЕРЕБР-Я':'СЕРЕБРЯНАЯ', 'ЮГУС':'ЮГУС', 
    'ЗАПОВЕДН ЗДОРОВЬЯ':'ЗАПОВЕДНИК ЗДОРОВЬЯ', 'ЛАЙФЛАЙН':'ЛАЙФЛАЙН', 'ЗАВЬЯЛОВ.':'ЗАВЬЯЛОВСКАЯ', 'SOC.MIN.Нап.VIT/GO':'Societe Minerale',

    # ... можно добавить другие бренды
}

def extract_brand(nomenclature, brands_dict):
    # Перебираем бренды в порядке убывания длины
    for brand in sorted(brands_dict.keys(), key=len, reverse=True):
        if brand.lower() in nomenclature.lower():
            return brands_dict[brand]
    return None  # или можно вернуть None

# Запуск функции
df_for_work['Бренд1'] = df_for_work['Номенклатура (вх)'].apply(lambda x: extract_brand(x, brands_mapping))

# Проверка возможных ошибок и вывод их в удобном формате
df_log_brand = df_for_work[df_for_work['Бренд1'].isnull()]
print(df_log_brand['Номенклатура (вх)'].value_counts())

## 5. Формируем справочник

In [ ]:
result = df_for_work[['Клиент_БарКод', 'Номенклатура (вх)', 'Продажи, шт.', 'Продажи в руб.',
       'Себестоимость в руб.', 'Клиент (вх)', 'Бар-код',
       'Подгруппа товара (вх)', 'Поставщик РЦ', 'Тип поставщика РЦ',
       'Производитель', 'Объем_л', 'Упаковка1', 'Бренд1', 'Газация']]

result.to_excel('spravochnik_sku.xlsx')

# II. Формирование справочника адресов

In [ ]:
# Отбираем необходимые поля для анализа в отдельный DataFrame
df_adress = df_enter[['Клиент (вх)', 'Код ТТ', 'Название ТТ', 'Адр', 'Формат ТТ (вх)', 
                      'Округ (вх)', 'Регион (вх)', 'Город (вх)', 'Адрес (вх)']]

# Оставляем только уникальные адреса
df_adress.drop_duplicates().reset_index(drop=True)

# Генерируем новые уникальное поле (ключ)
df_for_work['Клиент (вх)'] = df_for_work['Клиент (вх)'].str.strip()
df_for_work['Код ТТ'] = df_for_work['Код ТТ'].str.strip()
df_adress['Клиент_КодТТ'] = df_adress['Клиент (вх)'].str.cat(df_adress['Код ТТ'], sep='_')



## 1. Вытащим наименование города из поля с адресом

In [ ]:
def extract_city(address):
    if pd.isna(address):
        return None
    
    parts = [part.strip() for part in address.split(',')]
    
    # Список частей адреса в порядке приоритета (индексы: 2, 1, 3, 0, ...)
    priority_order = [2, 1, 3, 0] if len(parts) > 3 else [2, 1, 0]
    
    # Запрещённые маркеры (улицы, площади, набережные и т. д.)
    street_markers = [
        r'\bул\b|\bулица\b', r'\bпр-кт\b|\bпроспект\b', r'\bпер\b|\bпереулок\b',
        r'\bш\b|\bшоссе\b', r'\bб-р\b|\bбульвар\b', r'\bпл\b|\bплощадь\b',
        r'\bнаб\b|\bнабережная\b', r'\bпроезд\b', r'\bпр\b|\bпроезд\b',
        r'\bтер\b|\bтерритория\b', r'\bкм\b|\bкилометр\b', r'\bаллея\b',
        r'\bтракт\b', r'\bтупик\b', r'\bлиния\b', r'\bквартал\b', r'\bсад\b',
        r'\bпарк\b', r'\bмикрорайон\b', r'\bзона\b', r'\bучасток\b'
    ]
    street_pattern = re.compile('|'.join(street_markers), flags=re.IGNORECASE)
    
    for i in priority_order:
        if i >= len(parts):
            continue
        
        part = parts[i]
        
        # Пропускаем части с цифрами (например, "2А", "3-й")
        if re.search(r'\d', part):
            continue
        
        # Пропускаем улицы, площади, набережные и т. д.
        if street_pattern.search(part):
            continue
        
        # Очищаем от сокращений (г, с, р-н и т. д.)
        city = re.sub(
            r'\s*(г|с|п|пгт|р-н|рп|д|ст|х|ст-ца|кп|нп|мкр|тер|дп|кв-л)\s*$', 
            '', 
            part, 
            flags=re.IGNORECASE
        ).strip()
        
        if city:
            return city
    
    return None

# Запуск функции
df_adress["Город"] = df_adress["Адрес (вх)"].apply(extract_city)

# Проверка возможных ошибок и вывод их в удобном формате
df_log_city = df_adress[df_adress['Город'].isnull()]
print(df_log_city['Адрес (вх)'].value_counts())

## 2. Вытащим наименование региона РФ из поля с адресом

In [ ]:
def extract_region(address: str) -> Optional[str]:
    """
    Извлекает и нормализует название субъекта РФ из строки адреса.
    Возвращает None, если регион не удалось определить.
    """
    if pd.isna(address):
        return None
    
    # Приводим к строке и нормализуем пробелы
    address = str(address).replace('\n', ' ').replace('\t', ' ').strip()
    address = re.sub(r'\s+', ' ', address)
    
    # Исключаем обработку адресов с поселками (п) в начале
    if re.match(r'^[А-Яа-я]+\s+п,', address):
        return None
    
    # Варианты паттернов для разных форматов адресов
    patterns = [
        # Стандартный случай с индексом: "123456, Регион, ..."
        r'^\d{6},\s*([^,]+?)\s*(?:обл|край|Респ(?:ублика)?|АО|г|р-н|-\s*[^\s,]+)?(?:,|$)',
        # Случай с двойными запятыми: "123456,, Регион, ..."
        r'^\d{6},,\s*([^,]+?)\s*(?:обл|край|Респ(?:ублика)?|АО|г|р-н|-\s*[^\s,]+)?(?:,|$)',
        # Случай без индекса: "Регион, ..."
        r'^([^,]+?)\s*(?:обл|край|Респ(?:ублика)?|АО|г|р-н|-\s*[^\s,]+)(?:,|$)',
        # Случай с двойными запятыми в начале: ",, Регион, ..."
        r'^,,\s*([^,]+?)\s*(?:обл|край|Респ(?:ублика)?|АО|г|р-н|-\s*[^\s,]+)(?:,|$)'
    ]
    
    for pattern in patterns:
        match = re.search(pattern, address, re.IGNORECASE)
        if match:
            region = match.group(1).strip()
            
            # Удаляем лишние слова и сокращения
            region = re.sub(r'\s*г\.?$', '', region, flags=re.IGNORECASE)
            region = re.sub(r'\s*р-н$', '', region, flags=re.IGNORECASE)
            region = re.sub(r'\s*п$', '', region, flags=re.IGNORECASE)  # Удаляем "п" (поселок)
            
            # Полная нормализация названий
            region = normalize_region_name(region)
            
            return region if region else None
    
    return None

def normalize_region_name(region: str) -> str:
    """Нормализует название региона к стандартному виду"""
    if not region:
        return region
    
    # Специальные случаи
    special_cases = {
        'Санкт-Петербур': 'Санкт-Петербург',
        'Москва': 'Москва',
        'Санкт-Петербург': 'Санкт-Петербург',
        'Татарстан': 'Республика Татарстан',
        'Ханты-Мансийский': 'Ханты-Мансийский автономный округ - Югра',
        'Карелия': 'Республика Карелия',
        'Башкортостан': 'Республика Башкортостан',
        'Коми': 'Республика Коми',
        'Мордовия': 'Республика Мордовия',
        'Марий Эл': 'Республика Марий Эл',
        'Адыгея': 'Республика Адыгея',
        'Дагестан': 'Республика Дагестан',
        'Удмуртская': 'Удмуртская Республика',
        'Крым': 'Республика Крым',
        'Чувашия': 'Чувашская Республика',
        'Якутия': 'Республика Саха (Якутия)'
    }
    
    # Проверяем специальные случаи
    for key, value in special_cases.items():
        if key in region:
            return value
    
    # Нормализация областей
    oblast_matches = re.search(r'^(.*?)(\bобл\b|область)?$', region, re.IGNORECASE)
    if oblast_matches:
        base_name = oblast_matches.group(1).strip()
        if base_name.endswith('ая') or base_name.endswith('яя'):
            return f"{base_name} область"
    
    # Нормализация краев
    kray_matches = re.search(r'^(.*?)(\bкрай\b)?$', region, re.IGNORECASE)
    if kray_matches:
        base_name = kray_matches.group(1).strip()
        if base_name.endswith('ий') or base_name.endswith('ой'):
            return f"{base_name} край"
    
    # Нормализация республик
    resp_matches = re.search(r'^(.*?)(\bРесп\b|\bРеспублика\b)?$', region, re.IGNORECASE)
    if resp_matches and any(x in region for x in ['Респ', 'Республика']):
        base_name = resp_matches.group(1).strip()
        return f"Республика {base_name}" if not base_name.startswith('Республика') else base_name
    
    # Для случаев, когда ничего не подошло, возвращаем как есть
    return region

# Запуск функции
df_adress['Регион'] = df_adress['Адрес (вх)'].apply(extract_region)

# Проверка возможных ошибок и вывод их в удобном формате
df_log_region = df_adress[df_adress['Регион'].isnull()]
print(df_log_region['Адрес (вх)'].value_counts())

## 3. Создадим поле с указанием федерального округа на основе поля "Регион"

In [ ]:
def get_federal_district(region: str) -> Optional[str]:
    """
    Определяет федеральный округ по названию региона.
    Возвращает None, если регион не распознан.
    """
    if not region or pd.isna(region):
        return None
    
    # Словарь соответствия регионов федеральным округам
    federal_districts = {
        'Центральный федеральный округ': [
            'Москва', 'Белгородская область', 'Брянская область', 'Владимирская область',
            'Воронежская область', 'Ивановская область', 'Калужская область', 'Костромская область',
            'Курская область', 'Липецкая область', 'Московская область', 'Орловская область',
            'Рязанская область', 'Смоленская область', 'Тамбовская область', 'Тверская область',
            'Тульская область', 'Ярославская область'
        ],
        'Северо-Западный федеральный округ': [
            'Санкт-Петербург', 'Ленинградская область', 'Республика Карелия', 'Республика Коми',
            'Архангельская область', 'Вологодская область', 'Калининградская область', 'Мурманская область',
            'Новгородская область', 'Псковская область', 'Ненецкий автономный округ'
        ],
        'Южный федеральный округ': [
            'Республика Адыгея', 'Республика Калмыкия', 'Краснодарский край', 'Астраханская область',
            'Волгоградская область', 'Ростовская область', 'Севастополь', 'Республика Крым'
        ],
        'Северо-Кавказский федеральный округ': [
            'Республика Дагестан', 'Республика Ингушетия', 'Кабардино-Балкарская Республика',
            'Карачаево-Черкесская Республика', 'Республика Северная Осетия - Алания',
            'Чеченская Республика', 'Ставропольский край'
        ],
        'Приволжский федеральный округ': [
            'Республика Башкортостан', 'Республика Марий Эл', 'Республика Мордовия',
            'Республика Татарстан', 'Удмуртская Республика', 'Чувашская Республика', 'Чувашия',
            'Пермский край', 'Кировская область', 'Нижегородская область',
            'Оренбургская область', 'Пензенская область', 'Самарская область',
            'Саратовская область', 'Ульяновская область', 'Кемеровская область'
        ],
        'Уральский федеральный округ': [
            'Курганская область', 'Свердловская область', 'Тюменская область',
            'Челябинская область', 'Ханты-Мансийский автономный округ - Югра',
            'Ямало-Ненецкий автономный округ', 'Ямало-Ненецкий край'
        ],
        'Сибирский федеральный округ': [
            'Республика Алтай', 'Республика Тыва', 'Республика Хакасия', 'Алтайский край',
            'Красноярский край', 'Иркутская область', 'Кемеровская область - Кузбасс',
            'Новосибирская область', 'Омская область', 'Томская область', 'Забайкальский край'
        ],
        'Дальневосточный федеральный округ': [
            'Республика Саха (Якутия)', 'Камчатский край', 'Приморский край',
            'Хабаровский край', 'Амурская область', 'Магаданская область',
            'Сахалинская область', 'Еврейская автономная область', 'Чукотский автономный округ'
        ]
    }
    
    # Дополнительные альтернативные названия
    alternative_names = {
        'Республика Чувашская': 'Чувашская Республика',
        'Кемеровская область - Кузбасс': 'Кемеровская область',
        'Кабардино-Балкарская область': 'Кабардино-Балкарская Республика',
        'Северная Осетия - Алания': 'Республика Северная Осетия - Алания',
        'Карачаево-Черкесская область': 'Карачаево-Черкесская Республика',
        'Калмыкия': 'Республика Калмыкия',
        'Хакасия': 'Республика Хакасия',
        'Алтай': 'Республика Алтай',
        'Ингушетия': 'Республика Ингушетия'
    }
    
    # Заменяем альтернативные названия на стандартные
    region_std = alternative_names.get(region, region)
    
    # Ищем регион в каждом округе
    for district, regions in federal_districts.items():
        if region_std in regions:
            return district
    
    return None

# Запуск функции
df_adress['Округ'] = df_adress['Регион'].apply(get_federal_district)

# Проверка возможных ошибок и вывод их в удобном формате
df_log_fo = df_adress[df_adress['Округ'].isnull()]
print(df_log_fo['Адрес (вх)'].value_counts())

## 4. Создаем справочник адресов

In [ ]:
df_adress.to_excel('spravochnik_adress.xlsx')